In [29]:
from pynq import Overlay, MMIO
import pynq.lib as lib
import time
import numpy as np
from datetime import datetime
from pytz import timezone
overlay = Overlay("./SPI_GPIO_wrapper.bit")
AxiQspi = overlay.axi_quad_spi_0
overlay?

In [30]:
# Offset
# Interrupt Control Grouping
XSP_DGIER_OFFSET = 0x1C # Device global interrupt enable register
XSP_IISR_OFFSET = 0x20 # IP interrupt status register
XSP_IIER_OFFSET = 0x28 # IP interrupt enable register
# Core Grouping
XSP_SRR_OFFSET = 0x40 # Software reset register
XSP_CR_OFFSET = 0x60 # SPI control register
XSP_SR_OFFSET = 0x64 # SPI status register
XSP_DTR_OFFSET = 0x68 # SPI data transmit register.
XSP_DRR_OFFSET = 0x6C # SPI data receive register
XSP_SSR_OFFSET = 0x70 # SPI Slave select register
XSP_TFO_OFFSET = 0x74 # Transmit FIFO occupancy register
XSP_RFO_OFFSET = 0x78 # Receive FIFO occupancy register
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

# Mask
XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
## Crl mask
XSP_CR_TRANS_INHIBIT_MASK = 0x100 # Master Transaction Inhibit 0 = enable, 1 = disable
XSP_CR_LOOPBACK_MASK = 0x01 # Lookback mode: 0 = Normal operation, 1 = Loopback mode
XSP_CR_ENABLE_MASK = 0x02 # SPI system enable: 0 = SPI system disabled, 1 = SPI system enabled.
XSP_CR_MASTER_MODE_MASK = 0x04 # Set SPI mode: 0 = Slave configuration, 1 = Master configuration.
XSP_CR_CLK_POLARITY_MASK = 0x08 # Clock phase
XSP_CR_CLK_PHASE_MASK = 0x10 # Clock polarity
XSP_CR_TXFIFO_RESET_MASK = 0x20 # Transmit FIFO reset: 0 = Transmit FIFO normal operation, 1 = Reset transmit FIFO pointer.
XSP_CR_RXFIFO_RESET_MASK = 0x40 # Receive FIFO reset: 0 = Receive FIFO normal operation, 1 = Reset receive FIFO pointer.
XSP_CR_MANUAL_SS_MASK = 0x80 # Manual slave select assertion enable

SLAVE_NO_SELECTION = 0xFFFFFFFF

In [31]:
def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
    print("Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg) #?
    CRRead = AxiQspi.read(XSP_CR_OFFSET)
    print(f"XSP_CR_OFFSET : {bin(CRRead)[2:].zfill(10)}")
    return 0
print("Done")

Done


In [32]:
cnfg(AxiQspi)

Configure device
XSP_CR_OFFSET : 0110000110


0

In [33]:
def xfer(B2Send, B2Recv, AxiQspi):
    """
    SPI Transfer function that sends and receives data.

    Args:
        AxiQspi: SPI device object.
        B2Send: List of bytes to send.
        B2Recv: List to store received bytes.

    Returns:
        A list of received bytes.
    """

    print("TransferData")
    print(f"byte2Send: {B2Send}")
    StatusReg = AxiQspi.read(XSP_SR_OFFSET)
    print(f"STAT_REG (二進制) : {bin(StatusReg)[2:].zfill(11)}")
    for data in B2Send:
        print(f"data in B2Send: {bin(data)[2:].zfill(8)}")
        AxiQspi.write(XSP_DTR_OFFSET, data)
        AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

        StatusReg = AxiQspi.read(XSP_SR_OFFSET)
        while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
            StatusReg = AxiQspi.read(XSP_SR_OFFSET)
        RFORead=AxiQspi.read(XSP_RFO_OFFSET)    
        print(f"XSP_RFO_OFFSET : {bin(RFORead)[2:].zfill(8)}")
        #print('XSP_RFO_OFFSET : 0x{0:8b}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)

    print("ReadResponse")
    B2Recv.clear()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
#     print(f"RxFifoStatus (二進制) : {RxFifoStatus}")
    while RxFifoStatus == 0:
        temp = AxiQspi.read(XSP_RFO_OFFSET)
        print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET)
        print(f"XSP_DRR_OFFSET : {bin(temp)[2:].zfill(8)}")
        B2Recv.append(temp)
        print("temp: ",temp)
        print("B2Recv: ",B2Recv)
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    print("Done")
#     return resp
print("set xfer Done")

set xfer Done


In [34]:
def signedToUnsigned(n, byte_count):
    return int.from_bytes(n.to_bytes(byte_count, 'little', signed=True), 'little', signed=False)

def toSigned16(n):
    n = n & 0xffff
    return (n ^ 0x8000) - 0x8000

def getDateTime(): 
    format = "%Y-%m-%d %H:%M:%S"
    now_utc = datetime.now(timezone('UTC'))
    now_asia = now_utc.astimezone(timezone('Asia/Taipei'))
    print(now_asia.strftime(format))
    
# Read和Write(reg_addr) are DEX?y
# 要改，T和R的width都不一樣
def ReadIntan(reg_addr,times):
    hexReg=f"0x{reg_addr:02X}"
    basebit=0xC0
    bit2Send=basebit+reg_addr
    #print(bit2Send)
    byte2Send=[bit2Send,0x00]
    byte2Recv=[0x00,0x00,0x00,0x00]
    for num in range(1, times+1):
        xfer(byte2Send, byte2Recv, AxiQspi)
    #print(chr(byte2Recv[0]))
    MB=signedToUnsigned(byte2Recv[0], 1)
    LB=signedToUnsigned(byte2Recv[1], 1)
    realdata=MB<<8|LB
    #print(byte2Recv[1])
    return toSigned16(realdata)

# D要以2byte的方式傳送(hex)
def WriteIntan(R,D,timew):
    hexR=f"0x{R:02X}"
    wbasebit=0x80
    wbit2Send=wbasebit+R
    wbyte2Send=[wbit2Send,D]
    wbyte2Recv=[0x00,0x00,0x00,0x00]
    for num in range(1, timew+1):
        xfer(wbyte2Send, wbyte2Recv, AxiQspi)
    #print(wbyte2Recv)
    
def CaliIntan():
    cbyte2Send=[0x55,0x00]
    cbyte2Recv=[0x00,0x00,0x00,0x00]
    xfer(cbyte2Send, cbyte2Recv, AxiQspi)
    #print(cbyte2Recv)
    
def ConvertIntan(C):
    cobyte2Send=[C,0x00]
    cobyte2Recv=[0x00,0x00,0x00,0x00]
    xfer(cobyte2Send, cobyte2Recv, AxiQspi)
    #print(cobyte2Recv)
    MB=signedToUnsigned(cobyte2Recv[0], 1)
    LB=signedToUnsigned(cobyte2Recv[1], 1)
    realdata=MB<<8|LB
    return toSigned16(realdata)*CFactor

print("Done")

Done


In [35]:
#Check Intan connection and should get "INTAN" 5 characters
checklist=list(range(40,45))
for rom_addr in checklist:
    readreg=ReadIntan(rom_addr,3)
    print(chr(readreg),end='')

TransferData
byte2Send: [232, 0]
STAT_REG (二進制) : 00000100101
data in B2Send: 11101000
XSP_RFO_OFFSET : 00000000
data in B2Send: 00000000
XSP_RFO_OFFSET : 00000001
ReadResponse
XSP_RFO_OFFSET  : 0x00000001
XSP_DRR_OFFSET : 11111111
temp:  255
B2Recv:  [255]
XSP_RFO_OFFSET  : 0x00000000
XSP_DRR_OFFSET : 11111111
temp:  255
B2Recv:  [255, 255]
Done
TransferData
byte2Send: [232, 0]
STAT_REG (二進制) : 00000100101
data in B2Send: 11101000
XSP_RFO_OFFSET : 00000000
data in B2Send: 00000000
XSP_RFO_OFFSET : 00000001
ReadResponse
XSP_RFO_OFFSET  : 0x00000001
XSP_DRR_OFFSET : 11111111
temp:  255
B2Recv:  [255]
XSP_RFO_OFFSET  : 0x00000000
XSP_DRR_OFFSET : 11111111
temp:  255
B2Recv:  [255, 255]
Done
TransferData
byte2Send: [232, 0]
STAT_REG (二進制) : 00000100101
data in B2Send: 11101000
XSP_RFO_OFFSET : 00000000
data in B2Send: 00000000
XSP_RFO_OFFSET : 00000001
ReadResponse
XSP_RFO_OFFSET  : 0x00000001
XSP_DRR_OFFSET : 11111111
temp:  255
B2Recv:  [255]
XSP_RFO_OFFSET  : 0x00000000
XSP_DRR_OFFSET 